# 나이브 베이즈Naive Bayes
<hr/>


이 문서는 <a href="http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=84725482">밑바닥부터 시작하는 데이터 과학</a>[1]의 나이브 베이즈 단원에 대한 보충 설명이다. 책의 설명이 좀 부족하다 생각되어 보충 문서를 작성하게 되었다. 책에서 일반적인 나이브 베이즈 가정을 간결한 문장과 내용으로 잘 설명하고 있으나 베르누이 나이브 베이즈Bernoulli Naive Bayes에 대한 설명은 전혀 없는데 예제 코드는 이것을 구현하고 있다는데 있다. 다항분포 나이브 베이즈 모델과 베르누이 나이브 베이즈 모델을 잘 이해하기 위해서는 책에서 설명하는 나이브 베이즈 가정만으로는 부족하고 확률변수와 확률분포에 대한 이해가 조금 더 필요한데 여기에서 기본적으로 관련 내용을 간략하게 정리한 후 코드와 함께 살펴 보도록 한다. 책에서도 스팸메일 분류기를 예로 들어 설명하므로 여기서도 기본적으로 스펨 메일 분류를 예로 설명하도록 한다. 


## 1. 문제 정의
<hr/>

어떤 문서(또는 메세지) $d$가 있을 때 이 문서가 스팸($S$)클래스에 속하는지 일반 문서($ \neg S$)클래스에 속하는지 분류하는 문제가 있다고 하자. 그리고 우리에게 꽤 많은 단어들에 대해서 그 단어가 스팸메일에 나타날 확률과 일반메일에 나타날 확률에 대한 데이터를 가지고 있다고 하자. 예를 들면 "Sex는 일반메일에서 3%확률로 나타나고, 스팸메일에서 60%확률로 나타난다" 이런 데이터를 수만개 단어에 대해서 가지고 있다는 것이다. 기호로 적으면 $P(\text{'Naive'}|S)$와 $P(\text{'Naive'}|\neg S)$를 데이터로 가지고 있는 것이다. 우리가 풀고 싶은 문제는 $P(S\,|\,d)$ 와 $P(\neg S\,|\,d)$중 어느쪽이 더 큰가 하는 문제이다. $P(S\,|\,d)$ 는 문서 $d$가 주어졌다는 가정하에서 그 문서가 스팸일 확률 즉 조건부 확률을 나타낸다. $P(\neg S\,|\,d)$는 문서 $d$가 주어졌다는 가정하에서 그 문서가 스팸이 아닐 확률을 나타낸다.
해당 문제를 해결하기 위해 주어진 도큐먼트로 부터 판단에 사용할 특징feature를 추출해야 하는데 이때 어떤 확률분포를 사용하는지에 따라 특징벡터가 달라지게 된다. 일단 그 내용은 잠시뒤로 미루고 지금은 $d$로 부터 단어를 추출하여 특징 벡터 $\boldsymbol{x}=(x_{1}, x_{2}, ... , x_{n})$가 준비되었다하고 이야기를 계속 하자. 그럼 특징벡터 $\boldsymbol{x}$가 관찰되었을 때 그 벡터가 S에 속할 확률은 베이즈정리를 사용해서 아래와 같이 쓸 수 있다.

$$ P(S\,|\,x_{1}, x_{2}, ... , x_{n}) = \frac{P(x_{1}, x_{2}, ... , x_{n}\,|\,S)\,P(S)}{P(x_{1}, x_{2}, ... , x_{n})}$$

이때 특징벡터의 각 요소가 조건부 독립이라는 가정을 하면, 다시 말해 문서 $d$가 스팸이라는 가정하에 'love'이라는 단어가 나올 확률은 'partner'라는 단어가 나올 확률과 아무 상관이 없다고 가정하면 (실제로는 상관이 있을것이다. 그래서 이런 가정을 책[1]에서 '말도 안되는 가정'이라고 이야기한다.) 다음과 같이 다시 쓸 수 있다.

$$ P(S\,|\,x_{1}, x_{2}, ... , x_{n}) = \frac{P(x_{1}\,|\,S)\,P(x_{2}\,|\,S) \cdots P(x_{n}\,|\,S)\,P(S)}{P(x_{1})\,P(x_{2}) \cdots P(x_{n})}$$

조건부 독립가정하에서 $P(S\,|\, \boldsymbol{x})$와 $P( \neg S\,|\, \boldsymbol{x})$ 를 구하기 위해서는 $P(x_{1}\,|\,S)$, $P(x_{2}\,|\,S)$같은 것들을 알면되는데 이는 앞에 이미 우리는 이런 데이터를 엄청 많이 가지고 있다고 했다. 분모는 확률들의 곱이라 양수 인데 우리는 $P(S\,|\, \boldsymbol{x})$와 $P( \neg S\,|\, \boldsymbol{x})$의 크기 비교를 할것이므로 계산하지 않아도 무방하다. 즉 다음과 같다.

$$ P(S\,|\,x_{1}, x_{2}, ... , x_{n}) \propto P(x_{1}\,|\,S)\,P(x_{2}\,|\,S) \cdots P(x_{n}\,|\,S)\,P(S)$$

따라서 문서로부터 적절히 특징벡터를 추출하는 방법, 기존 데이터로 부터 $P(S\,|\, \boldsymbol{x})$와 $P( \neg S\,|\, \boldsymbol{x})$를 계산하는 방법, 어떤 문서가 스팸인지 아닌지를 나타내는 사전확률 $P(S)$, $P(\neg S)$을 구하는 방법을 알면 문제를 해결 할 수 있게 된다.


## 2. 실습
<hr/>

In [1]:
import os, math, random, re, glob
from collections import Counter, defaultdict

## 2.1.보조 함수 정의   

우선 실습에 필요한 보조함수를 정의한다. <span class="code-body">tokenize</span> 함수는 하나의 도큐먼트(메세지)를 단어별로 쪼갠 다음 중복을 제거하여 되돌리고, <span class="code-body">tokenize2</span>는 단어별로 쪼갠 다음 중복 단어가 있다면 그대로 허용한채 되돌린다. <span class="code-body">split_data</span>는 주어진 기존 데이터(학습데이터)를 학습용과 테스트용으로 분리하여 되돌린다.
(여기 사용된 코드들은 "밑바닥부터 시작하는 데이터 과학"의 코드를 대부분 이용하고 다항분포 나이브 베이즈 모델과 베르누이 나이브 베이즈 모델 구분하여 구현하기 위해 약간의 코드를 수정 추가했다.)


In [2]:
def split_data(data, prob):
    """
    split data into fractions [prob, 1 - prob]
    """
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

def tokenize(message):
    """
    주어진 메세지를 단어 단위로 자른 집합을 되돌림(단어의 중복제거)
    """
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return set(all_words) # remove duplicates

def tokenize2(message):
    """
    주어진 메세지를 단어 단위로 자른 집합을 되돌림(중복 단어 허용)
    """
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return all_words # duplicates

## 2.2. 데이터 로딩

"밑바닥부터 시작하는 데이터 과학"의 예제 소스 저장소에서 학습에 사용할 데이터 폴더 3개를 내려 받는다. 우선 3개 폴더에 있는 메일 데이터로 부터 제목을 추출하면 제목이 약 3423개가 추출이 된다. 이 3000여개의 도큐먼트에 대해 어떤 것이 스팸메일 제목인지 아닌지를 이미 다 알고 있다. 학습과 그 결과를 확인하기위해 이 3000여개의 데이터를 약 2500여개의 학습데이터와 800여개의 테스트 데이터로 나눈다.


In [3]:
#############################################################
# 데이터 만들기
# data = [ (메일의 제목, 스팸인가?아닌가? True or False), (), (), ... ]
#############################################################
data = []

#학습데이터는 스팸메일과 일반메일의 제목으로 한다.
subject_regex = re.compile(r"^Subject:\s+")

# 데이터가 들어있는 폴더 3개를 돌면서...
for fn in ['easy_ham' , 'hard_ham', 'spam']:
    is_spam = "ham" not in fn

    #디렉토리 내의 파일 리스트...
    for fn2 in glob.iglob(fn+"/*") :
        with open(fn2,'r',encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith("Subject:"):
                    subject = subject_regex.sub("", line).strip()
                    data.append((subject, is_spam))

print("전체 데이터 개수 : {} ".format(len(data)))
print("\n")

#############################################################
# data를 train_data, test_data로 나눔
#############################################################
random.seed(0)
train_data, test_data = split_data(data, 0.75)

print("학습 데이터 개수 : {} ".format(len(train_data)))

#############################################################
# 학습데이터 중에 스팸메일 개수와 일반메일 개수를 카운팅
#############################################################
num_spams = len([is_spam
                         for message, is_spam in train_data
                         if is_spam])
num_non_spams = len(train_data) - num_spams

print("학습데이터 중 스팸 메일 개수 : {} ".format(num_spams))
print("학습데이터 중 일반 메일 개수 : {} ".format(num_non_spams))
print("\n")

print("테스트 데이터 개수 : {} ".format(len(test_data)))

t_num_spams = len([is_spam
                         for message, is_spam in test_data
                         if is_spam])
t_num_non_spams = len(test_data) - t_num_spams
print("테스트데이터 중 스팸 메일 개수 : {} ".format(t_num_spams))
print("테스트데이터 중 일반 메일 개수 : {} ".format(t_num_non_spams))

전체 데이터 개수 : 3423 


학습 데이터 개수 : 2547 
학습데이터 중 스팸 메일 개수 : 364 
학습데이터 중 일반 메일 개수 : 2183 


테스트 데이터 개수 : 876 
테스트데이터 중 스팸 메일 개수 : 139 
테스트데이터 중 일반 메일 개수 : 737 


## 2.3. 학습  

### 2.3.1 사전확률 $P(S)$, $P(\neg S)$ 구하기

 앞서 적절히 특징벡터를 추출하는 방법, 기존 데이터로 부터 $P(S\,|\, \boldsymbol{x})$와 $P( \neg S\,|\, \boldsymbol{x})$를 계산하는 방법, 어떤 문서가 스팸인지 아닌지를 나타내는 사전확률 $P(S)$, $P(\neg S)$을 구하는 방법을 알면 분류기를 만들 수 있다고 했다. 사전확률 $P(S)$, $P(\neg S)$을 구하는 방법은 간단한다. 전체 도큐먼트 3000여개에 대해 이미 스팸, 햄(스펨이 아닌 경우를 가르킴)을 알고 았으므로 사전확률은 아래처럼 구할 수 았다.

$$ P(S) = \frac{\text{#spam}}{\text{#Document}} $$

$$ P(\neg S) = \frac{\text{#ham}}{\text{#Document}} $$


### 2.3.2 가능도Likelihood 구하기

나이브베이즈 분류기의 학습은 주어진 데이터를 바탕으로 가능도 $P(w_{i}\,|\,S)$, $P(w_{i}\,|\, \neg S)$를 계산하는 과정이다. 가능도는 학습데이터에 존재하는 모든 데이터에 대해서 구하게 된다. 문서의 앞에서 엄청 많은 단어에 대해 스펨클래스에서 그 단어가 나타날 확률을 이미 가지고 있다고 했는데 그 데이터를 만드는 작업을 하는 것이다. 일단 학습데이터에 존재하는 단어들을 사전에 담고 각 단어에 대해 스팸에서 출현한 횟수, 일반메일에서 출현한 횟수를 저장한다. 이때 베르누이 분포 모델을 쓸 경우 출현한 횟수가 많다 하더라도 1만 카운팅하고 다항분포 모델에서는 출현 수 만큼 그대로 카운팅한다. 이 자료를 단어장 $V$라 한다. 아래 코드가 있다.  


```python
for message, is_spam in train_data:
    """
    베르누이 분포 모델의 경우 워드카운팅 
    한 문서(message)에서 단어를 추출하고 중복을 제거한 후 그 유일 단어들에 대해서
    한번씩만 +1 을 함 결국 word가 해당 문서에 나왔다, 안나왔다만 표시
    """
    for word in tokenize(message): #중복을 제거한 단어들
        brn_counts[word][0 if is_spam else 1] += 1
    
    """
    다항 분포 모델의 경우 워드카운팅
    이번에는 중복을 제거하지 않고 카운팅을 함 그래서 한 문서에 word가 10번나오면
    10번 +1이 되서 결과적으로 해당 문서에 word가 몇번 나왔는지 저장하게 됨
    """
    for word in tokenize2(message): #중복을 제거하지 않은 단어들
        mtnm_counts[word][0 if is_spam else 1] += 1    
```


#### 2.3.2.1 베르누이 나이브 베이즈

우리의 단어장에 10개의 단어가 있다고 가정하자. 그리고 아래 그림처럼 그 단어중 하나를 임의로 뽑아 어떤 $C_{k}$에 속하는 주어진 도큐먼트 $d$에 나타나면 1, 나타나지 않으면 0을 가지는 확률변수 $X$가 있다 하자. 이 이산확률변수의 확률분포는 베르누이 분포가 되고 만약 $X$의 대응 관계가 아래 그림과 같다면 확률질량함수는 $ P(X=x) = p^{x}(1-p)^{1-x} $이다. 여기서 $x$는 $w_i$가 $d$에 나타나면 1, 안나타나면 0을 가지는 바이너리 변수이고 $p=\frac{4}{10}$이다.  

<img src="1.png" width="350">  

나이브 베이즈에서 정의되는 확률변수는 위의 경우와 조금 다르다. 각 $w_{i}$마다 확률 변수를 하나씩 대응시켜 아래 그림과 같이 정의 한다.  

<img src="2.png" width="600">  

이 경우 각 확률변수 $X_i$에 대해 $ P(X_{i}=x) = p_{i}^{x}(1-p_{i})^{1-x} $이고  주어진 도큐먼트는 $C_k$에 속하는 도큐먼트라 했으므로 여기서 $p_{i}$라 함은 $w_i$가 $C_k$에 나타날 확률이 된다.  $p_{i}=P(w_{i}\,|\,C_{k})$이다. 이제 $P(w_{i}\,|\,C_{k})$를 어떻게 구할 것인가 생각해봐야 한다. (우리 문제에서는 $k=2$인 경우로 $C_{1}=S$, $C_{2}= \neg S $ 로 생각하면 됨) 특정 단어 $w_{i}$가 문서에 나타날 확률분포는 자연계에 분명 존재할 것이다. 우리가 그것을 정확히 모를 뿐이다. $w_{i}$가 제약이 없이 그냥 문서에 나타날 확률(엄밀히 말하면 문서라는것도 일종의 제약이다)과 특정 제약이 있는 예를 들어 스팸문서에 나타날 확률은 엄연히 다를 것이다. 우리는 메일중에서 스팸메일에 나타날 확률, 일반메일에 나타날 확률을 알면 되므로 $P(w_{i}\,|\,S)$, $P(w_{i}\,|\,\neg S)$를 알고 싶은 것이다. 자연상태에 존재하는 이 확률분포를 정확히 알 방법은 없다. 따라서 우리가 가지고 있는 데이터를 바탕으로 근사된 $\hat{P}(w_{i}\,|\,S)$, $\hat{P}(w_{i}\,|\,\neg S)$ 구하여야 한다.

베르누이 모델에서 단어장 $V$ 중 특정 단어 $w_i$가 특정 문서 클래스에 나타날 확률은

<img src="7.png">

로 계산한다. 즉, 그 단어가 한 문서에서 얼마나 자주 나타났는지는 관심없고 나타났는지 안나타났는지만 관심이 있는것이다.
예를 들어 총 10개의 문서가 있는 C라는 클래스에서 naive라는 단어가 1번 문서에 3번, 2번 문서에 5번 나오고 3번~10번 문서에는 나오지 않은 상황이라면 
베르누이 모델의 경우 $\hat{P}(\text{'naive'}|C) = \frac{2}{10}$이 된다.

#### 2.3.2.2 다항분포  나이브 베이즈

반면 다항분포 모델에서는 단어가 출현하는 횟수도 중요하므로

<img src="8.png">

로 계산한다.  

예를 들어 총 10개의 문서가 있는 C라는 클래스에서 naive라는 단어가 1번 문서에 3번, 2번 문서에 5번 나오고 3번~10번 문서에는 나오지 않고, 10개의 문서에 $V$에 포함된 단어가 총 50개 쓰였고, 각 단어의 출현수를 모두 합하면 200번이라 하자.
다항분포 모델의 경우는 $\hat{P}(\text{'naive'}|C) = \frac{8}{200} $이 된다.

### 2.3.3 스무딩 적용

아래 코드 결과를 보면 3888개의 단어에 대해 베르누이 모델을 위해 특정 단어가 나타나는 스팸메일의 수, 특정 단어가 나타나는 일반메일의 수, 다항분포 모델을 위해 그 단어가 스팸메일에 나타나는 출현 수와 일반메일에 나타나는 출현 수를 알게 되었다. 이 후 그 정보를 이용하여 $P(w_{i}\,|\,S)$, $P(w_{i}\,|\, \neg S)$ 계산해서 저장을 하는데 이 때 라플라스 스무딩laplace smoothing을 적용해야 한다. 스무딩 적용 이유는 책[1]에 잘 나와있기 때문에 별도로 언급하지 않고 어떤 원리로 스무딩을 적용하는지 두가지 경우에 대해 나눠서 설명한다.

#### 2.3.3.1 베르누이 나이브 베이즈

스무딩의 목적이 가능도가 0이 되는 것을 방지하기 위함이라 분자에 적당한 수 k를 더해서 0이되는 것을 막는 것인데 그냥 분자에만 k를 더하면 안된다. 다시 한번 아래 그림을 보고 베르누이 분포 모델에서 가능도를 구하는 방법을 생각해보자.

<img src="3.png" width="600">

전체 스팸메일 수가 분모에 가고 분자에 특정 단어를 포함하는 스팸메일 수가 온다. 문제는 어떤 특정 단어가 일반메일에만 나타나고 스팸메일에는 한번도 안나타나는 경우 가능도는 0이 되고 그 단어가 포함된 스팸메일은 절대로 스팸으로 분류되지 않는다는 것이다. 이런 현상을 막기위해 분자에 임의의 수 k를 더해준다. 그리고, 당연하게도 분모는 그 단어가 나타나는 스팸메일과 일반메일의 합이므로 일반메일 그룹에도 k를 더해준다. 따라서 베르누이 분포 모델의 경우 가능도는 아래와 같이 구한다. (결국 분모에는 2k가 더해짐)

<img src="4.png" width="600">

#### 2.3.3.2 다항분포 나이브 베이즈

반면 다항분포 모델에서는 가능도가 아래 그림처럼 단어의 출현수로 계산이 된다. 그래서 특정 단어가 스팸메일에 0번 출현했다면 (출현하지 않았다면) 같은 문제가 생기게 된다.

<img src="5.png" width="600">

이때도 역시 분자에 k를 더해 k번 출현한 것으로 가정한다. 이는 우리가 모은 스팸메일 데이터 이전에 우리가 모으지 못한 어떤 스팸메일 군에서 k번 출현했을 것이다 라고 가정하는 것이다. 그렇다면 당연히 다른 단어들에도 k번 이미 출현했다고 가정해야 하므로 분모의 모든 단어의 출현수에 k를 더해 준다. 이 경우 분모에는 (k $\times$ '스팸에 나온 단어의 개수') 가 더해지게 된다.

<img src="6.png" width="600">

아래 코드는 설명한 스무딩을 적용해서 가능도를 구하는 과정이다.

```python
"""
베르누이 분포를 사용하는 모델의 경우
spam : w가 포함된 스팸문서 수
num_spam : 스팸문서 수
분자에 k, 분모에 2k 더함
결과 : [(단어1, P(단어1|S), P(단어1|ㄱS), ... (단어n, P(단어n|S), P(단어n|ㄱS)]
"""
k = 0.5 #smoothing factor
brn_word_probs =[(w,
             (spam + k) / (num_spams + 2*k), 
             (non_spam + k) / (num_non_spams + 2*k))
                for w, (spam, non_spam) in brn_counts.items()]

"""
다항 분포를 사용하는 모델의 경우
spam : w의 스팸메일 출현 수
Vfs  : V에 있는 단어들이 스팸메일에 나타난 총 출현 수
Vs   : |V| 즉, 단어장에 유일단어 수
분자에 k, 분모에 k*|V| 더함
결과 : [(단어1, P(단어1|S), P(단어1|ㄱS), ... (단어n, P(단어n|S), P(단어n|ㄱS)]
"""
k = 0.1
mtnm_word_probs =[(w,
             (spam     + k) / (Vfs + k*Vs),
             (non_spam + k) / (Vfh + k*Vh))
                for w, (spam, non_spam) in mtnm_counts.items()]
```

코드 결과에서 확인할 수 있듯이 3888개의 단어에 대해 위 작업을 수행하였다.


In [4]:
"""
학습 목표 : P(w|S), P(w,|¬S) 를 만든다. 
먼저 단어를 각 메일에서 나온 횟수를 카운트한다. 
counts := dict('word1':[스팸메일에 나온 수, 일반메일에 나온 수], 'word2':[스팸메일에 나온 수, 일반메일에 나온 수], ... )
"""
#############################################################
# 사전확률 P(S), P(¬S) 를 계산 
#############################################################
prior_S = num_spams / (num_spams+num_non_spams)
prior_non_S = num_non_spams / (num_spams+num_non_spams)

print("클래스 사전확률")
print("P(S)  : {:f} ".format(prior_S))
print("P(¬S) : {:f} ".format(prior_non_S))
print("\n")

all_words = {'spam':[], 'ham':[]}
brn_counts  = defaultdict(lambda: [0, 0])
mtnm_counts = defaultdict(lambda: [0, 0])
for message, is_spam in train_data:
    for word in tokenize(message):
        brn_counts[word][0 if is_spam else 1] += 1
    
    for word in tokenize2(message):
        mtnm_counts[word][0 if is_spam else 1] += 1    
        if is_spam :
            all_words['spam'].append(word)
        else:
            all_words['ham'].append(word)

Vfs = len(all_words['spam'])
Vfh = len(all_words['ham'])
    
Vs = len(set(all_words['spam']))
Vh = len(set(all_words['ham']))

print("스팸메일에 있는 유일 단어 총수 : {}".format(Vs))
print("그 단어들의 총 출연수(중복포함) : {}".format(Vfs))
print("\n")
print("일반메일에 있는 유일 단어 총수 : {}".format(Vh))      
print("그 단어들의 총 출연수(중복포함) : {}".format(Vfh))
print("\n")

"""
turn the word_counts into a dictionary of triplets 
word_probs = {word1:(p(word1|S), p(word1|¬S)) , word2:(p(word2|S), p(word2|¬S)), ... }
"""
k = 0.1 #smoothing factor
brn_word_probs =[(w,
             (spam + k) / (num_spams + 2 * k),
             (non_spam + k) / (num_non_spams + 2 * k))
                for w, (spam, non_spam) in brn_counts.items()]

k = 0.1
mtnm_word_probs =[(w,
             (spam     + k) / (Vfs + k*Vs),
             (non_spam + k) / (Vfh + k*Vh))
                for w, (spam, non_spam) in mtnm_counts.items()]


print("학습 단어장의 크기:{}".format(len(brn_counts)))

클래스 사전확률
P(S)  : 0.142913 
P(¬S) : 0.857087 


스팸메일에 있는 유일 단어 총수 : 1023
그 단어들의 총 출연수(중복포함) : 2288


일반메일에 있는 유일 단어 총수 : 3289
그 단어들의 총 출연수(중복포함) : 13294


학습 단어장의 크기:3888


## 2.4. 분류

주어진 메시지에 대한 스팸 여부를 판별한다. 학습데이터에 있는 모든 단어들에 대해 $P(w_{i}\,|\,S)$, $P(w_{i}\,|\, \neg S)$를 계산해두었으므로 주어진 메세지를 단어로 분리한 후 특징 벡터로 만들고 모든 단어의 가능도 값을 곱한다. 


### 2.4.1 베르누이 나이브 베이즈

#### 2.4.1.1 특징 벡터 만들기


베르누이 분포 모델에서는 $V$의 단어가 주어진 $d$에 나왔는지 안나왔는지만 판단하므로 다음의 예와 같이 특징벡터를 만든다.

```python
V = [w1, w2, w3, w4, w5, w6, w7, w8]  #학습데이터로 부터 추출된 단어장
d = "w2 w5 w2 w7 w5 w5"
x = [0, 1, 0, 0, 1, 0, 1, 0] 
```

V의 각 단어가 d에 나왔는지 판단 확률변수 $X_1=0$, $X_2=1$, $X_3=0$, $X_4=0$, $X_5=1$, $X_6=0$, $X_7=1$, $X_8=0$ 처럼 된다.

#### 2.4.1.2 가능도 계산

이 후 아래 수식을 이용하여 가능도를 곱해 나간다.

$$ 
\begin{align}
P(C_{k} \,|\, \boldsymbol{x}) &\propto P(\boldsymbol{x} \,|\, C_{k})P(C_{k}) \\
&\propto P(C_{k}) \prod_{i=1}^{|V|} p(w_{i}\,|\,C_{k})^{x_i} ( 1-p(w_{i}\,|\,C_{k}))^{1-x_{i}} 
\end{align}
$$

여기서, $\boldsymbol{x}$는 입력된 메세지가 변환된 특징벡터, $|V|$은 학습된 전체단어의 개수이고, $x_{i}$는 $\boldsymbol{x}$의 요소이며 학습된 단어장의 i번째 단어인 $w_{i}$가 입력된 메세지에 포함되면 1 없으면 0을 가지는 바이너리 변수이다. 수식에 모든 요소들은 이미 다 앞에서 계산해 둔 것들이다. 그리고 책에서는 사전확률 $P(S)$, $P(\neg S)$를 곱하지 않는데 여기서는 명시적으로 곱해주었다. 구체적인 코드는 아래와 같다.

```python
#입력된 메세지를 단어별로 자른다. 여기선 잘린 단어의 중복 허용안함
message_words = tokenize(message)

# 클래스 사전확률을 곱해주고 P(S)*∏P(w|S) , P(¬S)*∏P(w|¬S)
log_prob_if_spam ,log_prob_if_not_spam = math.log(prior_S),math.log(prior_non_S)

for word, prob_if_spam, prob_if_not_spam in brn_word_probs:
    # for each word in the message,
    # add the log probability of seeing it
    if word in message_words:
        log_prob_if_spam += math.log(prob_if_spam)
        log_prob_if_not_spam += math.log(prob_if_not_spam)

    # for each word that's not in the message
    # add the log probability of _not_ seeing it
    else:
        log_prob_if_spam += math.log(1.0 - prob_if_spam)
        log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
```

### 2.4.2 다항분포 나이브 베이즈

#### 2.4.2.1 특징 벡터 만들기

다항분포 모델에서는 $V$의 단어가 주어진 $d$에 몇번 나왔는지 판단하므로 다음의 예와 같이 특징벡터를 만든다.

```python
V = [w1, w2, w3, w4, w5, w6, w7, w8]  #학습데이터로 부터 추출된 단어장
d = "w2 w5 w2 w7 w5 w5"
x = [0, 2, 0, 0, 3, 0, 1, 0]
```

V의 각 단어가 d에 몇번 나왔는지 판단 하므로 확률변수는 $X_1=0$, $X_2=2$, $X_3=0$, $X_4=0$, $X_5=3$, $X_6=0$, $X_7=1$, $X_8=0$ 처럼 된다.

#### 2.4.2.2 가능도 계산

이 후 아래 수식을 이용하여 가능도를 곱해 나간다.

$$
\begin{align}
P(C_{k}\,|\,\boldsymbol{x} ) &\propto P(\boldsymbol{x} \,|\, C_{k})P(C_{k}) \\
&\propto P(C_{k}) \prod_{i=1}^{|V|} P(w_{i}\,|\,C_{k})^{x_i} 
\end{align}
$$

여기서, $\boldsymbol{x}$는 입력된 메세지가 변환된 특징벡터, $|V|$은 학습된 전체단어의 개수이고, $x_{i}$는 $\boldsymbol{x}$의 요소이며 학습된 단어장의 i번째 단어인 $w_{i}$가 입력된 메세지에 나타난 출현수이다. 역시 수식에 모든 요소들은 이미 다 앞에서 계산해 둔 것들이다. 구현에 있어서 한가지 팁이 있는데 어차피 단어장 $V$의 단어중 $d$에 한번도 나타나지 않은 단어의 가능도는 $P(w_{i}\,|\,C_{k})^{0}=1$이 되므로 구현에서는 $d$에 속하는 단어의 가능도만 곱하면 된다. 구체적인 코드는 아래와 같다.  

```python
#입력된 메세지를 단어별로 자른다. 여기선 잘린 단어의 중복 허용
message_words = tokenize2(message)

# 클래스 사전확률을 곱해주고 P(S)*∏P(w|S) , P(¬S)*∏P(w|¬S)
log_prob_if_spam ,log_prob_if_not_spam = math.log(prior_S),math.log(prior_non_S)
    
for word, prob_if_spam, prob_if_not_spam in mtnm_word_probs:
    # for each word in the message,
    if word in message_words:
        # ∏p(w|S) , ∏p(w|¬S) -> ∑ log(p(w|S)) , ∑ log(p(w|¬S))    
        log_prob_if_spam += math.log(prob_if_spam)
        log_prob_if_not_spam += math.log(prob_if_not_spam)   
```


In [5]:
###############################################################################
def brn_spam_probability(message):    
    """
    책에 코드는 설명은 기본적인 multinomial 나이브베이즈를 설명하고 코드는
    베르누이 나이브 베이즈로 작성 되어 있다.
    https://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html
    https://www.datascienceschool.net/view-notebook/c19b48e3c7b048668f2bb0a113bd25f7/
    그러면서 아무런 언급이 없음.
    """
    global prior_S, prior_non_S
    
    message_words = tokenize(message)
    log_prob_if_spam ,log_prob_if_not_spam = math.log(prior_S),math.log(prior_non_S)
    #log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    for word, prob_if_spam, prob_if_not_spam in brn_word_probs:
        # for each word in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

def mtnm_spam_probability(message):
    """
    주어진 메세지가 스팸인지 아닌지 학습된 likelihood 로부터 계산한다.
    일반적인 multinomial 나이브베이즈로 작성.
    """
    global prior_S, prior_non_S
   
    message_words = tokenize2(message)
    
    # 클래스 사전확률을 곱해주고 P(S)*∏P(w|S) , P(¬S)*∏P(w|¬S)
    log_prob_if_spam ,log_prob_if_not_spam = math.log(prior_S),math.log(prior_non_S)
    
    for word, prob_if_spam, prob_if_not_spam in mtnm_word_probs:
        # for each word in the message,
        if word in message_words:
            # ∏p(w|S) , ∏p(w|¬S) -> ∑ log(p(w|S)) , ∑ log(p(w|¬S))    
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)    
        
    prob_if_spam     = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)

    return prob_if_spam / (prob_if_spam + prob_if_not_spam) 
###############################################################################

#테스트 데이터를 분류한다.
brn_classified = [(subject, is_spam, brn_spam_probability(subject))
              for subject, is_spam in test_data]

mtnm_classified = [(subject, is_spam, mtnm_spam_probability(subject))
              for subject, is_spam in test_data]

#분류된 결과를 성공 실패로 나눠서 카운팅한다.
counts = Counter((is_spam, spam_prob > 0.5) # (actual, predicted)
                for _, is_spam, spam_prob in brn_classified)

print("베르누이 나이브 베이즈")
print("일반 메일을 일반 메일이라고 분류한 경우 : {}".format(counts[(False,False)]))
print("일반 메일을 스팸 메일이라고 분류한 경우 : {}".format(counts[(False,True)]))
print("스팸 메일을 스팸 메일이라고 분류한 경우 : {}".format(counts[(True,True)]))
print("스팸 메일을 일반 메일이라고 분류한 경우 : {}".format(counts[(True,False)]))
print("정밀도 : {:f}".format( counts[(True,True)] / (counts[(True,True)]+counts[(False,True)]) ) )
print("재현율 : {:f}".format( counts[(True,True)] / (counts[(True,True)]+counts[(True,False)]) ) )
print("\n")

counts = Counter((is_spam, spam_prob > 0.5) # (actual, predicted)
                for _, is_spam, spam_prob in mtnm_classified)

print("다항분포 나이브 베이즈")
print("일반 메일을 일반 메일이라고 분류한 경우 : {}".format(counts[(False,False)]))
print("일반 메일을 스팸 메일이라고 분류한 경우 : {}".format(counts[(False,True)]))
print("스팸 메일을 스팸 메일이라고 분류한 경우 : {}".format(counts[(True,True)]))
print("스팸 메일을 일반 메일이라고 분류한 경우 : {}".format(counts[(True,False)]))
print("정밀도 : {:f}".format( counts[(True,True)] / (counts[(True,True)]+counts[(False,True)]) ) )
print("재현율 : {:f}".format( counts[(True,True)] / (counts[(True,True)]+counts[(True,False)]) ) )


베르누이 나이브 베이즈
일반 메일을 일반 메일이라고 분류한 경우 : 708
일반 메일을 스팸 메일이라고 분류한 경우 : 29
스팸 메일을 스팸 메일이라고 분류한 경우 : 108
스팸 메일을 일반 메일이라고 분류한 경우 : 31
정밀도 : 0.788321
재현율 : 0.776978


다항분포 나이브 베이즈
일반 메일을 일반 메일이라고 분류한 경우 : 698
일반 메일을 스팸 메일이라고 분류한 경우 : 39
스팸 메일을 스팸 메일이라고 분류한 경우 : 114
스팸 메일을 일반 메일이라고 분류한 경우 : 25
정밀도 : 0.745098
재현율 : 0.820144


두 경우 모두에서 코드 몇줄로는 썩 나쁘지 않은 결과를 나타내었음을 확인할 수 있다.


## 3. 참고문헌
<hr/>

1. 밑바닥부터 시작하는 데이터 과학Data Science from Scratch, 한빛미디어, Orelly, 조엘 그루스

2. Introduction to Information Retrieval, Cambridge University Press. 2008. (Chapter 13 Text classiﬁcation and Naive Bayes) https://nlp.stanford.edu/IR-book/

3. Lecture Note for Informatics, Univ. of Edinburgh http://www.inf.ed.ac.uk/teaching/courses/inf2b/lectureSchedule.html


In [1]:
%%html
<link href='https://fonts.googleapis.com/earlyaccess/nanummyeongjo.css' rel='stylesheet' type='text/css'>
<link href='http://fonts.googleapis.com/earlyaccess/nanumgothiccoding.css' rel='stylesheet' type='text/css'>
<style>
    p  { font-family: 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%;  text-indent: 10px; }
    li { font-family: 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%; }
    li > p { text-indent: 0px; }
    code, pre { font-family: 'Nanum Gothic Coding' !important; }
    
    .code-body { font-family: 'Nanum Gothic Coding'; }
</style>